1. Installing the required libraries and importing the relevant libraries

In [9]:
# Installing the required libraries
!pip install openpyxl

# Importing the relevant libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


2. Mounting the Google Drive

In [10]:
# Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3. Loading the Excel File

In [11]:
# Loading the Excel file
file_path = '/content/drive/MyDrive/CM2603 Data Science Group Project/Data Regardings/IMPORT STATISTICS - 2023.xlsx'
excel_data = pd.ExcelFile(file_path)

# Inspecting the Sheet Names
print("Sheet Names:", excel_data.sheet_names)

Sheet Names: ['Dataset_1', 'Dataset_2', 'Dataset_3', 'Dataset_4', 'Dataset_5', 'Dataset_6', 'Dataset_7', 'Dataset_8', 'Dataset_9', 'Dataset_10', 'Dataset_11', 'Dataset_12', 'Dataset_13', 'Dataset_14', 'Dataset_15', 'Dataset_16', 'Dataset_17', 'Dataset_18', 'Dataset_19', 'Dataset_20', 'Dataset_21', 'Dataset_22', 'Dataset_23', 'Dataset_24', 'Dataset_25', 'Dataset_26', 'Dataset_27', 'Dataset_28', 'Dataset_29', 'Dataset_30', 'Dataset_31', 'Dataset_32', 'Dataset_33', 'Dataset_34', 'Dataset_35', 'Dataset_36', 'Dataset_37', 'Dataset_38', 'Dataset_39', 'Dataset_40', 'Dataset_41', 'Dataset_42', 'Dataset_43', 'Dataset_44', 'Dataset_45', 'Dataset_46', 'Dataset_47', 'Dataset_48', 'Dataset_49', 'Dataset_50', 'Dataset_51', 'Dataset_52', 'Dataset_53', 'Dataset_54', 'Dataset_55', 'Dataset_56', 'Dataset_57', 'Dataset_58', 'Dataset_59', 'Dataset_60', 'Dataset_61', 'Dataset_62', 'Dataset_63', 'Dataset_64', 'Dataset_65', 'Dataset_66', 'Dataset_67', 'Dataset_68', 'Dataset_69', 'Dataset_70', 'Dataset_71', '

4. Loading the data from all the existing sheets

In [12]:
# Loading Data from All Sheets (Process only the first sheet as an example)
data = excel_data.parse(excel_data.sheet_names[0])  # First sheet
print(f"\nLoaded Sheet: {excel_data.sheet_names[0]}")

# Inspecting Data
print("\nFirst 5 rows of the dataset:")
print(data.head())
print("\nDataset Info:")
data.info()
print("\nMissing Values Summary:")
print(data.isnull().sum())


Loaded Sheet: Dataset_1

First 5 rows of the dataset:
                    IMPORTER  YEAR      MONTH  HSCODE      COUNTRY UNIT  \
0       TransWorld Chemicals  2023       JUNE  280110       FRANCE  Kg.   
1  Horizon Industrial Supply  2023  SEPTEMBER  283719         U.K.  Kg.   
2     Greenfield Enterprises  2023       JULY  284700  SOUTH KOREA  Kg.   
3       Redwood Chemical Co.  2023   DECEMBER  280110      AUSTRIA  Kg.   
4       Redwood Chemical Co.  2023      APRIL  280800        SPAIN  Kg.   

   QUANTITY  VALUE_RS                                     DESCRIPTION_01  \
0    179640  13387839  Inorganic chemicals; Organic or inorganic comp...   
1     53784  36871839  Inorganic chemicals; Organic or inorganic comp...   
2     77028  35228981  Inorganic chemicals; Organic or inorganic comp...   
3    137434  23393445  Inorganic chemicals; Organic or inorganic comp...   
4    116881  11490994  Inorganic chemicals; Organic or inorganic comp...   

                                     

5. Cleaning the column names

In [13]:
# Cleaning column names
data.columns = [col.strip().upper().replace(" ", "_") for col in data.columns]
data.head()


,IMPORTER,YEAR,MONTH,HSCODE,COUNTRY,UNIT,QUANTITY,VALUE_RS,DESCRIPTION_01,DESCRIPTION_02,DESCRIPTION_03
0,TransWorld Chemicals,2023,JUNE,280110,FRANCE,Kg.,179640,13387839,Inorganic chemicals; Organic or inorganic comp...,"Fluorine, chlorine, bromine and iodine","Fluorine, chlorine, bromine and iodine"
1,Horizon Industrial Supply,2023,SEPTEMBER,283719,U.K.,Kg.,53784,36871839,Inorganic chemicals; Organic or inorganic comp...,"Cyanides, cyanide oxides and complex cyanides.","Cyanides, cyanide oxides and complex cyanides."
2,Greenfield Enterprises,2023,JULY,284700,SOUTH KOREA,Kg.,77028,35228981,Inorganic chemicals; Organic or inorganic comp...,"Hydrogen peroxide, whether or not solidified w...","Hydrogen peroxide, whether or not solidified w..."
3,Redwood Chemical Co.,2023,DECEMBER,280110,AUSTRIA,Kg.,137434,23393445,Inorganic chemicals; Organic or inorganic comp...,"Fluorine, chlorine, bromine and iodine","Fluorine, chlorine, bromine and iodine"
4,Redwood Chemical Co.,2023,APRIL,280800,SPAIN,Kg.,116881,11490994,Inorganic chemicals; Organic or inorganic comp...,Nitric acid; sulphonitric acids.,Nitric acid; sulphonitric acids.


6. Handling the missing values

In [14]:
# Filling missing categorical values with "UNKNOWN"
categorical_cols = ["IMPORTER", "MONTH", "HSCODE", "COUNTRY", "UNIT",
                    "DESCRIPTION_01", "DESCRIPTION_02", "DESCRIPTION_03"]
for col in categorical_cols:
    data[col] = data[col].fillna("UNKNOWN")


In [15]:
# Filling missing numerical values
numerical_cols = ["YEAR", "QUANTITY", "VALUE_RS"]
for col in numerical_cols:
    if col == "YEAR":
        data[col] = data[col].fillna(data[col].mode()[0] if not data[col].mode().empty else 2023)
    elif col == "QUANTITY":
        data[col] = data[col].fillna(data[col].median())
    else:
        data[col] = data[col].fillna(0)


7. Standardize Text Values

In [18]:
# Defining the categorical columns
categorical_cols = ["IMPORTER", "MONTH", "COUNTRY", "UNIT", "DESCRIPTION_01", "DESCRIPTION_02", "DESCRIPTION_03"]

# Standardize the text columns
for col in categorical_cols:
    data[col] = data[col].fillna("").astype(str).str.strip().str.upper()



8. Convert Data Types

In [19]:
# Converting data types for numeric columns
data["YEAR"] = pd.to_numeric(data["YEAR"], errors="coerce").fillna(2023).astype(int)
data["QUANTITY"] = pd.to_numeric(data["QUANTITY"], errors="coerce").fillna(0).astype(int)
data["VALUE_RS"] = pd.to_numeric(data["VALUE_RS"], errors="coerce").fillna(0).astype(float)


9. Handle Outliers

In [20]:
# Handling outliers in numerical columns
for col in ["QUANTITY", "VALUE_RS"]:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[col] = np.clip(data[col], lower_bound, upper_bound)


10. Feature Engineering

In [21]:
# Creating a VALUE_PER_UNIT feature
data["VALUE_PER_UNIT"] = data["VALUE_RS"] / data["QUANTITY"].replace(0, np.nan)



In [22]:
# Mapping months to numeric values
month_mapping = {
    "JANUARY": 1, "FEBRUARY": 2, "MARCH": 3, "APRIL": 4, "MAY": 5, "JUNE": 6,
    "JULY": 7, "AUGUST": 8, "SEPTEMBER": 9, "OCTOBER": 10, "NOVEMBER": 11, "DECEMBER": 12
}
data["MONTH_NUMBER"] = data["MONTH"].map(month_mapping).fillna(0).astype(int)


In [23]:
# Categorizing importers based on total value
data["IMPORTER_CATEGORY"] = pd.qcut(data["VALUE_RS"], q=4, labels=["Low", "Medium", "High", "Very High"])


11. Encode Categorical Variables

In [24]:
# Encoding the categorical variables
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le


12. Scale Numerical Data

In [25]:
# Scale numerical data
scaler = MinMaxScaler()
numerical_cols_to_scale = ["QUANTITY", "VALUE_RS", "VALUE_PER_UNIT"]
data[numerical_cols_to_scale] = scaler.fit_transform(data[numerical_cols_to_scale])


13. Final Integration

In [26]:
# Save preprocessed data
output_file = "Advanced_Preprocessed_Import_Statistics_2023.xlsx"
data.to_excel(output_file, index=False)
print(f"Advanced preprocessed data saved to {output_file}")


Advanced preprocessed data saved to Advanced_Preprocessed_Import_Statistics_2023.xlsx
